<a href="https://colab.research.google.com/github/Yusuf80216/my_notebooks/blob/main/Natural_Language_Processing_with_Disaster_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!gdown 1l9aitkOcA13c-8bme2yJ8b0XSlMmNM1o

Downloading...
From: https://drive.google.com/uc?id=1l9aitkOcA13c-8bme2yJ8b0XSlMmNM1o
To: /content/nlp-getting-started.zip
100% 607k/607k [00:00<00:00, 123MB/s]


In [17]:
!unzip 'nlp-getting-started.zip'

Archive:  nlp-getting-started.zip
  inflating: sample_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: test.csv                
  inflating: train.csv               


# METHOD 1

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
trainData = pd.read_csv('train.csv')

In [ ]:
trainData

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


In [ ]:
len(trainData)

7613

In [ ]:
trainData.isnull().sum()

text      0
target    0
dtype: int64

In [ ]:
trainData = trainData.drop(columns = ['id', 'keyword', 'location'])

In [ ]:
trainData.sample(5)

,text,target
6662,WHITE AMERICANS ARE THE BIGGEST THREAT IN THE ...,0
2533,Hey girl you must be Toe Hobbit: Part Two: ghe...,0
3305,So all the store's fire alarms went off today ...,1
849,Rip ?? Blood !,0
2210,China is only delaying the deluge: If the fund...,0


In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
ps = PorterStemmer()
corpus = []
vocab_size = 5000

for i in range(0, len(trainData)):
    tweet = re.sub('[^a-zA-Z]', ' ', trainData['text'][i])
    tweet = tweet.lower()
    tweet = tweet.split()
    tweet = [ps.stem(word) for word in tweet if not word in stopwords.words('english')]
    tweet = ' '.join(tweet)
    corpus.append(tweet)

In [ ]:
onehot_repr = [one_hot(words, vocab_size) for words in corpus]

In [ ]:
corpus[1]

'forest fire near la rong sask canada'

In [ ]:
onehot_repr[1]

[621, 95, 3186, 895, 242, 1935, 3832]

In [ ]:
trainData['text'][1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0, 3475, 3348, 1610, 2223, 1264, 4961,
       3501,  812, 3795, 4941, 1386], dtype=int32)

In [ ]:
# To get the maximum length of the any text
length = []
for i in range(0, len(onehot_repr)):
    length.append(len(onehot_repr[i]))

print(max(length))

27


In [ ]:
sent_len = 27
embedded_docs = pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_len)

In [ ]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,  621,   95,
       3186,  895,  242, 1935, 3832], dtype=int32)

In [ ]:
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()

# trainData['location'] = le.fit_transform(trainData['location'])
# trainData['keyword'] = le.fit_transform(trainData['keyword'])

Data Splitting

In [ ]:
X = np.array(embedded_docs)
y = np.array(trainData['target'])

In [ ]:
# trainData['text'] = [np.array(lst) for lst in trainData['text']]

In [ ]:
# X = X.astype(np.float32)
# y = y.astype(np.int32)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 1)

Model Selection

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout

In [ ]:
embedding_vector_features = 50

model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features, input_length = sent_len))
model.add(LSTM(200))
# model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

ValueError: ignored

In [ ]:
model.fit(X_train, y_train, validation_data = [X_test, y_test], batch_size = 32, epochs = 10)

Epoch 1/10
191/191 [==============================] - 19s 85ms/step - loss: 0.5635 - accuracy: 0.7082 - val_loss: 0.4666 - val_accuracy: 0.7866
Epoch 2/10
191/191 [==============================] - 16s 84ms/step - loss: 0.3670 - accuracy: 0.8417 - val_loss: 0.4837 - val_accuracy: 0.7741
Epoch 3/10
191/191 [==============================] - 15s 81ms/step - loss: 0.2759 - accuracy: 0.8929 - val_loss: 0.6068 - val_accuracy: 0.7380
Epoch 4/10
191/191 [==============================] - 16s 81ms/step - loss: 0.1936 - accuracy: 0.9263 - val_loss: 0.6906 - val_accuracy: 0.7466
Epoch 5/10
191/191 [==============================] - 16s 81ms/step - loss: 0.1243 - accuracy: 0.9560 - val_loss: 0.9005 - val_accuracy: 0.7446
Epoch 6/10
191/191 [==============================] - 16s 84ms/step - loss: 0.0764 - accuracy: 0.9765 - val_loss: 1.0650 - val_accuracy: 0.7229
Epoch 7/10
191/191 [==============================] - 15s 80ms/step - loss: 0.0626 - accuracy: 0.9793 - val_loss: 1.2503 - val_accuracy:

In [ ]:
y_pred = model.predict(X_test)

48/48 [==============================] - 1s 20ms/step


In [ ]:
y_pred = (y_pred > 0.5)

In [ ]:
len(y_pred)

1523

In [ ]:
from sklearn.metrics import accuracy_score
acc = accuracy_score(y_test, y_pred)
acc

0.7235718975705844

# METHOD 2

In [5]:
import numpy as np
import pandas as pd

In [6]:
trainData = pd.read_csv('train.csv')
testData = pd.read_csv('test.csv')

In [7]:
from sklearn import feature_extraction, model_selection, linear_model, preprocessing

In [8]:
countVectorizer = feature_extraction.text.CountVectorizer()

In [11]:
trainVectors = countVectorizer.fit_transform(trainData['text'])

testVectors = countVectorizer.transform(testData['text'])

In [12]:
clf = linear_model.RidgeClassifier()

In [13]:
scores = model_selection.cross_val_score(clf, trainVectors, trainData['target'], cv=3, scoring='f1')

scores

array([0.59453669, 0.5642787 , 0.64082434])

In [15]:
clf.fit(trainVectors, trainData['target'])

RidgeClassifier()

In [19]:
sample_submission = pd.read_csv('/content/sample_submission.csv')

In [21]:
sample_submission['target'] = clf.predict(testVectors)

In [22]:
sample_submission.head()

,id,target
0,0,0
1,2,1
2,3,1
3,9,0
4,11,1


In [23]:
sample_submission.to_csv('NLP with Disaster Tweets.csv', index=False)